In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import healpy as hp
import numpy as np
from scipy import optimize
import pandas as pd

In [ ]:
beta=hp.ang2vec(264.021,48.253,lonlat=True) #direction of CMB dipole

The newt cell reads data from the optimization, for all fluxes, and all samples. The first four arrays of each list correspond to the lowest flux, the second four to the second lowest and so on.

In [ ]:
full=['full','wo0_5','noMo']                        
mask=['maskfullz','maskwo0_5','masknoMo']                  
flux=['1','5','10','']
#from full-sky simulations
amp_f=[]            #amplitudes
dire_f=[]       #direction vector
dire_f_lb=[]    #direction in galactic coordinates
#from masked-sky simulations
amp_m=[]
dire_m=[]
dire_m_lb
for f in flux:
    for n in full:
        amp_f.append(np.array(pd.read_csv('path/simul%s/%s/amp.dat'%(f,n),header=None,sep=' ')))
        dire_f.append(np.array(pd.read_csv('path/simul%s/%s/dir_vec.dat'%(f,n),header=None,sep=' ')))
        dire_f_lb.append(np.array(pd.read_csv('path/simul%s/%s/dir_lonlat.dat'%(f,n),header=None,sep=' ')))
        if n=='noMo':
            amp_f.append(np.array(pd.read_csv('path/simul%s/structure/amp_deg_full.dat'%f,header=None,sep=' ')))
            pix=np.array(pd.read_csv('path/simul%s/structure/dir_deg_full.dat'%f,header=None,sep=' '))
            vec=[]
            for p in pix:
                vec.append(hp.pix2vec(32,p,nest=True))
            dire_f.append(np.array(vec))
            lon,lat=hp.pix2ang(32,pix,nest=True,lonlat=True)
            dire_f_lb.append(np.array(np.hstack([lon,lat])))
    for n in mask:
        amp_m.append(np.array(pd.read_csv('path/simul%s/%s/amp12.dat'%(f,n),header=None,sep=' ')))
        dire_m.append(np.array(pd.read_csv('path/simul%s/%s/dir22.dat'%(f,n),header=None,sep=' ')))
        dire_m_lb.append(np.array(pd.read_csv('path/simul%s/%s/dir22_lonlat.dat'%(f,n),header=None,sep=' ')))                 
        if mas=='maskwo0_5noMo':
            amp_m.append(np.array(pd.read_csv('path/simul%s/structure/amp_deg_mask.dat'%f,header=None,sep=' ')))
            pix=np.array(pd.read_csv('path/simul%s/structure/dir_deg_mask.dat'%f,header=None,sep=' '))
            vec=[]
            for p in pix:
                vec.append(hp.pix2vec(32,p,nest=True))
            dire_m.append(np.array(vec))
            lon,lat=hp.pix2ang(32,pix,nest=True,lonlat=True)
            dire_m_lb.append(np.array(np.hstack([lon,lat])))

In [ ]:
#separate the directional parameters in galactic longitud and latitude
l_f,b_f,l_m,b_m=[],[],[],[]
for i in range(len(dire_f_lb)):
    l_f.append(dire_f_lb[i].T[:][0])
    b_f.append(dire_f_lb[i].T[:][1])
    l_m.append(dire_m_lb[i].T[:][0])
    b_m.append(dire_m_lb[i].T[:][1])

In [ ]:
#normalize direction vectors
dirnorm_f,dirnorm_m=[],[]
for d in range(len(dire_f)):
    vec_f,vec_m=[],[]
    for v in dire_f[d]:
        vec_f.append(v/np.linalg.norm(v))
    dirnorm_f.append(vec_f)
    for a in dire_m[d]:
        vec_m.append(a/np.linalg.norm(a))
        dirnorm_m.append(vec_m)

In [ ]:
#calculate angle between estimated direction and CMB direction
ang_f,ang_m=[],[]
for m in range(len(dirnorm_f)):
    ang=[]
    for d in dirnorm_f[m]:
        cos=np.dot(beta,d)
        ang.append(np.arccos(cos))
    ang_f.append(np.array(ang))
    ang=[]
    for d in dirnorm_m[m]:
        cos=np.dot(beta,d)
        ang.append(np.arccos(cos))
    ang_m.append(np.array(ang))

The following cell prints the mean of the amplitude (times 1e3) for each flux threshold and sample.
The parameter of which the mean is wanted to be obtained can be changed.

In [ ]:
pm=u"\u00B1"    #define plus-minus sign
for i in amp_m:
    print(str(round(np.mean(i*1000),2))+pm+str(round(np.std(i*1000),2)))

Below different models of how the plots have been obtained are presented.

The following cell creates the amplitude histograms.

In [ ]:
masks=['S>1 '+uJy,'S>5 '+uJy,'S>10 '+uJy,'S>22.8 '+uJy]
fig, ax=plt.subplots(2,2,figsize=(14,10))
fig.subplots_adjust(wspace=0.2,hspace=0.35)
c=0
for j in range(2):
    for i in range(2):
        kin=amp_m[0+4*c]
        wol=amp_m[1+4*c]
        nomo=amp_m[2+4*c]
        struc=amp_m[3+4*c]
        ax[j][i].set_title(masks[c],fontsize=24)    
        ax[j][i].hist(struc,bins=np.arange(0,0.007,0.0003),edgecolor='black',fc=(0,1,0,0.5),lw=0.5)
        ax[j][i].hist(nomo,bins=np.arange(0,0.007,0.0003),edgecolor='black',fc=(1,1,0,0.5),lw=0.5)
        ax[j][i].hist(kin,bins=np.arange(0,0.007,0.0003),edgecolor='black',fc=(0,0,1,0.5),lw=0.5)
        ax[j][i].hist(wol,bins=np.arange(0,0.007,0.0003),edgecolor='black',fc=(1,0,0,0.5),lw=0.5)
        ax[j][i].axvline(0.00462,c='black',lw=4)
        ax[j][i].tick_params(axis='both', which='major', labelsize=14)
        ax[j][i].set_ylabel('counts',fontsize=18)
        ax[j][i].set_xlabel('amplitude',fontsize=18)
        c+=1
fig.savefig('amplitude_hist.pdf',bbox_inches='tight')

The next model has been used for direction scatter plots of all samples.
In the for loop a transformation is made in the parameters to fit this modules coordinate intervals:
 $l=[ -\pi , \pi ]$ and $b=[ -\pi/2 , \pi/2 ]$
for the structure part another the same transformation has to be done but it is more complex because of the
randomness of the directions.

In [ ]:
fig=plt.figure(figsize=(8,6))
ax=fig.add_subplot(111, projection="mollweide")
la,ba=[],[]
for i in range(2):
    la.append(np.deg2rad(360-l_m[i]))
    ba.append(np.deg2rad(b_m[i]))
ax.scatter(l_m[3],b_m[3],c='g',marker='.',label='structure')
ax.scatter(la[0],ba[0],c='b',marker='.',label='kinematic')
ax.scatter(la[1],ba[1],c='r',marker='.',label='kinematic w/o local structure')
ax.scatter(np.deg2rad(360-264.021),np.deg2rad(48.253),c='black',marker='o',label='CMB')
ax.grid()
ax.set_xlabel(r"$l$",fontsize=20)
ax.legend()
h=ax.set_ylabel(r'$b$',fontsize=20)
h.set_rotation(0)
ax.set_title('S>1 '+uJy,fontsize=28)
ax.set_xticklabels([150,120,90,60,30,0,330,300,270,240,210,180])
ax.tick_params(axis='both', which='major', labelsize=14)
fig.savefig('mollweide_dirs.pdf',bbox_inches='tight')

Longitude, latitude and theta histograms have been obtained following the next model.

In [ ]:
colors=[(0,0,1,0.5),(1,0,0,0.5),(1,1,0,0.5),(0,1,0,0.5)]
fig, ax=plt.subplots(3,4,figsize=(24,20))
ax[0][0].set_title(r'$S\!>1\,\mu$Jy',fontsize=26)
ax[0][1].set_title(r'$S\!>5\,\mu$Jy',fontsize=26)
ax[0][2].set_title(r'$S\!>10\,\mu$Jy',fontsize=26)
ax[0][3].set_title(r'$S\!>22.8\,\mu$Jy',fontsize=26)
for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        li=l[i+4*j]
        if i!=3:
            ax[i][j].hist(li,bins=np.arange(220,300,10),edgecolor='black',fc=cols[i],lw=0.5)
            ax[i][j].axvline(np.mean(li),c='brown',lw=3)
            ax[i][j].axvline(264.02,c='black',lw=3)
        else:
            ax[i][j].hist(li,bins=np.arange(0,360,45),edgecolor='black',fc=cols[i],lw=0.5)
            ax[i][j].axvline(np.mean(li),c='brown',lw=3,label=str(round(np.mean(li),1)))
            ax[i][j].legend(fontsize=15)
        ax[i][j].tick_params(axis='both', which='major', labelsize=18)
        if j==0:
            ax[i][j].set_ylabel('counts',fontsize=22)
        if i==3:
            ax[i][j].set_xlabel(r'$\l$ / deg',fontsize=22)
fig.savefig('longitude.pdf',bbox_inches='tight')